In [1]:
# default_exp data

# data

> Read data from netdata rest api into a pandas dataframe.

In [2]:
# hide
from nbdev.showdoc import *

In [3]:
# hide
# export
import asks
import trio
import pandas as pd
import requests
from netdata_pandas.wrangle import drop_low_uniqueness_cols, drop_low_std_cols

In [4]:
# export


def get_chart_list(host: str = '127.0.0.1:19999', starts_with: str = None) -> list:
    """Get list of all available charts on a `host`.  
    
    ##### Parameters:  
    - **host** `str` The host we want to get a list of available charts from.
    - **starts_with** `str` A string to filter the list of charts returns to just those that start with `starts_with`.
    
    ##### Returns:  
    - **chart_list** `list` A list of availalbe charts.
    
    """
    
    url = f"http://{host}/api/v1/charts"
    r = requests.get(url)
    charts = r.json().get('charts')
    chart_list = [chart for chart in charts]
    if starts_with:
        chart_list = [chart for chart in chart_list if chart.startswith(starts_with)]
    return chart_list

In [5]:
# tests

# get some charts from london demo site
charts = get_chart_list('london.my-netdata.io', starts_with='system.')

# check just system. charts returned
assert set([chart.split('.')[0] for chart in charts]) == set(['system'])


In [6]:
# export


async def get_chart(api_call: str, data: list, col_sep: str ='|'):
    """Get data for an individual chart.
    
    ##### Parameters:  
    - **api_call** `tuple` A tuple of (`url`,`chart`) for the url to pull data from and chart it represents.
    - **data** `list` A list for dataframes for each chart to be appended to.
    - **col_sep** `str` A character for separating chart and dimension in column names of dataframe.
    
    """
    
    url, chart, host = api_call
    r = await asks.get(url)
    r_json = r.json()
    df = pd.DataFrame(r_json['data'], columns=['time_idx'] + r_json['labels'][1:])
    df['host'] = host
    df = df.set_index(['host','time_idx']).add_prefix(f'{chart}{col_sep}')
    data.append(df)

In [7]:
# export


async def get_charts(api_calls: list, col_sep: str ='|', timeout: int = 60) -> pd.DataFrame:
    """Create a nursey to make seperate async calls to get each chart.
    
    ##### Parameters:  
    - **api_calls** `list` A list of tuple's of [(`url`,`chart`),...] of api calls that need to be made.
    - **col_sep** `str` A character for separating chart and dimension in column names of dataframe.
    - **timeout** `int` The number of seconds for trio to [move_on_after](https://trio.readthedocs.io/en/stable/reference-core.html#trio.move_on_after).
    
    ##### Returns:  
    - **df** `pd.DataFrame` A pandas dataframe with all chart data outer joined based on time index.
    
    """
    
    data = []
    with trio.move_on_after(timeout):
        async with trio.open_nursery() as nursery:
            for api_call in api_calls:
                nursery.start_soon(get_chart, api_call, data, col_sep)
    df = pd.concat(data, join='outer', axis=1, sort=True)
    return df

In [8]:
# export


def get_data(hosts: list = ['london.my-netdata.io'], charts: list = ['system.cpu'], after: int = -60, 
             before: int = 0, points: int = 0, col_sep: str = '|', numeric_only: bool = False,
             ffill: bool = True, diff: bool = False, timeout: int = 60, nunique_thold = None, 
             std_thold: float = None, index_as_datetime: bool = False, freq: str = 'infer', group: str = 'average') -> pd.DataFrame:
    """Define api calls to make and any post processing to be done.
    
    ##### Parameters:  
    - **hosts** `list` A list of hosts to pull data from.
    - **charts** `list` A list of charts to pull data for.
    - **after** `int` The timestamp or relative integer from which to pull data after.
    - **before** `int` The timestamp or relative integer from which to pull data before.
    - **points** `int` The `points` parameter to pass to the api call if need to aggregate data in some way.
    - **col_sep** `str` A character for separating chart and dimension in column names of dataframe.
    - **numeric_only** `bool` Set to true if you want to filter out any non numeric data.
    - **ffill** `bool` Set to true if you want to forward fill any null or missing values.
    - **diff** `bool` Set to true if you want to get the difference of metrics as opposed to their raw value.
    - **timeout** `int` The number of seconds for trio to [move_on_after](https://trio.readthedocs.io/en/stable/reference-core.html#trio.move_on_after).
    - **nunique_thold** [`float`,`int`] If defined calls function to filter cols with low number of unique values.
    - **std_thold** `float` If defined calls function to filter cols with low standard deviation.
    - **index_as_datetime** `bool` If true, set the index to be a pandas datetime.
    - **freq** `str` Freq to be passed to pandas datetime index.
    - **group** `str` The grouping function to use.
    
        
    ##### Returns:  
    - **df** `pd.DataFrame` A pandas dataframe with all chart data outer joined based on time index and any post processing done.
    
    """
    
    # if hosts is a string make it a list of one
    if isinstance(hosts, str):
        hosts = [hosts]
        
    # if charts is a string make it a list of one
    if isinstance(charts, str):
        # if specified get all charts
        if charts == 'all':
            charts = get_chart_list(hosts[0])
        else:
            charts = [charts]
        
    
    # define list of all api calls to be made
    api_calls = [
        (f'http://{host}/api/v1/data?chart={chart}&after={after}&before={before}&points={points}&format=json&group={group}', chart, host)
        for host in hosts for chart in charts
    ] 
    # get the data
    df = trio.run(get_charts, api_calls, col_sep, timeout)
    # post process the data
    if len(hosts) == 1:
        df = df.reset_index(level=0, drop=True)
    if numeric_only:
        df = df._get_numeric_data()
    if ffill:
        df = df.ffill()
    if diff:
        df = df.diff().dropna(how='all')
    if nunique_thold:
        df = drop_low_uniqueness_cols(df, nunique_thold)
    if std_thold:
        df = drop_low_std_cols(df, std_thold)
    if index_as_datetime:
        df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.index, unit='s'), freq=freq))
    return df

In [9]:
# examples

df = get_data('london.my-netdata.io', 'all', after=-60, before=0, nunique_thold=0.05)
print(df.shape)
print(df.head())

(62, 561)
            web_log_nginx.requests_per_ipproto|ipv4  \
time_idx                                              
1601567878                                14.785464   
1601567879                                15.000283   
1601567880                                16.854500   
1601567881                                12.362195   
1601567882                                 9.216069   

            web_log_nginx.requests_per_ipproto|ipv6  \
time_idx                                              
1601567878                                 0.072620   
1601567879                                 2.782863   
1601567880                                 2.999870   
1601567881                                 0.217268   
1601567882                                 0.000000   

            nginx_local.connections_accepted_handled|accepted  \
time_idx                                                        
1601567878                                                NaN   
1601567879             

In [10]:
# tests

# get some test data
test_host = 'london.my-netdata.io'
test_charts = ['system.cpu', 'system.load']
df = get_data(test_host, test_charts, after=-60, before=0, col_sep='|')

# look for some expected columns
assert 'system.load|load1' in df.columns
assert 'system.cpu|user' in df.columns
# check expected shape of data
assert str(df.shape) == '(60, 12)' or '(61, 12)'
# check that all types are float64 or int64
assert len(df.dtypes[df.dtypes != 'int64'][df.dtypes != 'float64']) == 0

# test index as datetime
df = get_data('london.my-netdata.io', 'system.cpu', index_as_datetime=True)
assert isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex)
assert isinstance(df.index.freq, pd.tseries.offsets.Second)

In [11]:
# export


def get_alarm_log(host: str = '127.0.0.1:19999', datetimes: bool = True) -> pd.DataFrame:
    """Get alarm log from `host`.  
    
    ##### Parameters:  
    - **host** `str` The host we want to get the alarm log from.
    
    ##### Returns:  
    - **df** `pd.DataFrame` A df of the alarm_log.
    
    """
    
    url = f"http://{host}/api/v1/alarm_log"
    r = requests.get(url)
    alarm_log = r.json()
    df = pd.DataFrame(alarm_log)
    if datetimes:
        for col in ['when', 'delay_up_to_timestamp']:
            df[col] = pd.to_datetime(df[col], unit='s')
    return df



In [12]:
# tests 

df = get_alarm_log('london.my-netdata.io')

expected_cols = ['hostname', 'unique_id', 'alarm_id', 'alarm_event_id', 'name', 'chart', 'family', 'processed', 'updated', 'exec_run', 'exec_failed', 'exec', 'recipient', 'exec_code', 'source', 'units', 'when', 'duration', 'non_clear_duration', 'status', 'old_status', 'delay', 'delay_up_to_timestamp', 'updated_by_id', 'updates_id', 'value_string', 'old_value_string', 'last_repeat', 'silenced', 'info', 'value', 'old_value', 'no_clear_notification']

assert list(df.columns) == expected_cols
assert len(df) >= 1

In [13]:
# tests

# test grouping behaves as expected

# get some test data
test_host = 'london.my-netdata.io'
test_charts = ['system.load']

# get raw data
df_last100 = get_data(test_host, test_charts, after=-100, before=0, col_sep='|')

# direclty get aggregations
df_avg = get_data(test_host, test_charts, after=-100, before=0, col_sep='|', points=1, group='average')
df_std = get_data(test_host, test_charts, after=-100, before=0, col_sep='|', points=1, group='stddev')
df_min = get_data(test_host, test_charts, after=-100, before=0, col_sep='|', points=1, group='min')
df_max = get_data(test_host, test_charts, after=-100, before=0, col_sep='|', points=1, group='max')

# calc by hand
df_last100_avg = df_last100.mean()
df_last100_std = df_last100.std()
df_last100_min = df_last100.min()
df_last100_max = df_last100.max()

# get diffs
avg_diffs = round(abs(df_avg - df_last100_avg), 2)
std_diffs = round(abs(df_std - df_last100_std), 2)
min_diffs = round(abs(df_min - df_last100_min), 2)
max_diffs = round(abs(df_max - df_last100_max), 2)

# assert abs differences are small
tolerance = 0.1
assert (avg_diffs <= tolerance).values.tolist() == [[True, True, True]]
assert (std_diffs <= tolerance).values.tolist() == [[True, True, True]]
assert (min_diffs <= tolerance).values.tolist() == [[True, True, True]]
assert (max_diffs <= tolerance).values.tolist() == [[True, True, True]]